In [20]:
import tensorflow as tf
import numpy as np
import cv2

In [21]:
class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(28*28, activation='sigmoid'),
        ])

    def call(self, inputs):
        return self.fc(inputs)

In [22]:
class Discriminator(tf.keras.Model):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc = tf.keras.Sequential([
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(1, activation='linear'),
        ])

    def call(self, inputs):
        return self.fc(inputs)

In [23]:
class WGAN_GP():
    def __init__(self):
        self.noise_size = 16
        self.generator = Generator()
        self.discriminator = Discriminator()
        self.generator.build(input_shape=(None, self.noise_size))
        self.discriminator.build(input_shape=(None, 28*28))
        self.g_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
        self.d_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

    def train(self, dataset, batch_size=1024, epochs=500):
        for e in range(epochs):
            generator_loss = list()
            discriminator_loss = list()
            for i in range(int(len(dataset) / batch_size)):
                real_image = dataset[i * batch_size: (i+1)*batch_size]
                normal_z = np.random.normal(size=(batch_size, self.noise_size))
                with tf.GradientTape() as tape:
                    d_loss = self.d_loss(self.generator, self.discriminator, normal_z, real_image)
                grads = tape.gradient(d_loss, self.discriminator.trainable_variables)
                self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_variables))
                with tf.GradientTape() as tape:
                    g_loss = self.g_loss(self.generator, self.discriminator, normal_z)
                grads = tape.gradient(g_loss, self.generator.trainable_variables)
                self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_variables))
                generator_loss.append(g_loss)
                discriminator_loss.append(d_loss)
            g_l = np.mean(generator_loss)
            d_l = np.mean(discriminator_loss)
            print("epoch: {}/{}, generator loss: {}, discriminator loss: {}".format(e+1, epochs, g_l, d_l))
            if (e+1) % 10 == 0:
                self.save_image("./WGAN-GP/image_epochs_{}.png".format(e+1))
    
    def save_image(self, path):
        normal_z = np.random.normal(size=(1, self.noise_size))
        image = self.generator.predict(normal_z)
        image = np.reshape(image, newshape=(28, 28)) * 255.0
        cv2.imwrite(path, image)

    @staticmethod
    def gradient_penalty(discriminator, real_image, fake_image):
        assert real_image.shape[0] == fake_image.shape[0]
        batch_size = real_image.shape[0]
        eps = tf.random.uniform([batch_size, 1])
        inter = eps * real_image + (1. - eps) * fake_image
        with tf.GradientTape() as tape:
            tape.watch([inter])
            d_inter_logits = discriminator(inter)
        grads = tape.gradient(d_inter_logits, inter)
        grads = tf.reshape(grads, [grads.shape[0], -1])
        gp = tf.norm(grads, axis=1)
        gp = tf.reduce_mean((gp - 1.) ** 2)
        return gp
    
    @staticmethod
    def g_loss(generator, discriminator, noise_z):
        fake_image = generator(noise_z)
        d_fake_logits = discriminator(fake_image)
        loss = - tf.reduce_mean(d_fake_logits)
        return loss

    @staticmethod
    def d_loss(generator, discriminator, noise_z, real_image):
        fake_image = generator(noise_z)
        d_fake_logits = discriminator(fake_image)
        d_real_logits = discriminator(real_image)
        gp = WGAN_GP.gradient_penalty(discriminator, real_image, fake_image)
        loss = tf.reduce_mean(d_fake_logits) - tf.reduce_mean(d_real_logits) + 10. * gp
        return loss, gp
    

In [24]:
dataset = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = dataset.load_data()
x_train = np.reshape(x_train, newshape=(-1, 28*28)) / 255.0

gen = WGAN_GP()
gen.train(dataset=x_train)

epoch: 1/500, generator loss: 4.7165374755859375, discriminator loss: -2.106144666671753
epoch: 2/500, generator loss: 0.4603469967842102, discriminator loss: -1.6288366317749023
epoch: 3/500, generator loss: 0.1497013121843338, discriminator loss: -1.8890736103057861
epoch: 4/500, generator loss: 0.2547372579574585, discriminator loss: -1.6553547382354736
epoch: 5/500, generator loss: 0.2565060555934906, discriminator loss: -1.6102776527404785
epoch: 6/500, generator loss: 0.11520905047655106, discriminator loss: -1.421623945236206
epoch: 7/500, generator loss: 0.32349395751953125, discriminator loss: -1.3993253707885742
epoch: 8/500, generator loss: 0.2909322679042816, discriminator loss: -1.3377583026885986
epoch: 9/500, generator loss: 0.06489350646734238, discriminator loss: -1.2587331533432007
epoch: 10/500, generator loss: 0.40025752782821655, discriminator loss: -1.1577603816986084
1/1 [==============================] - 0s 32ms/step
epoch: 11/500, generator loss: 0.227708280086